In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim

from datetime import datetime
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from tqdm import tqdm 

In [2]:
current_date = datetime.now().strftime("%Y-%m-%d")
pth_file_path = f'./eeg_emotion-best_model_{current_date}.pth'
onnx_file_path = f"./eeg_emotion-best_model_{current_date}.onnx"

In [4]:
class CustomDataset(Dataset):
    def __init__(self, data_frame):
        self.features = data_frame.iloc[:, :-1].values
        self.labels = data_frame.iloc[:, -1].values

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        features = torch.FloatTensor(self.features[idx])
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        return features, label

def shuffle_data(train_df, test_df):
    # Shuffle the training and testing dataframes
    train_df_shuffled = train_df.sample(frac=1).reset_index(drop=True)
    test_df_shuffled = test_df.sample(frac=1).reset_index(drop=True)
    
    return train_df_shuffled, test_df_shuffled


class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(140, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 3)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        x = torch.softmax(x, dim=1)
        return x

def train_model(model, train_loader, test_loader, num_epochs=50, learning_rate=0.001, save_path=pth_file_path):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        test_accuracy, _ = evaluate_model(model, test_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Test Accuracy: {test_accuracy}")

        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            torch.save(model.state_dict(), save_path)
            print("Model saved!")

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    pred_list = []
    real_list = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            pred_list.extend(predicted.cpu().numpy())
            real_list.extend(labels.cpu().numpy())
    accuracy = correct / total
    cm = confus_matrix(real_list, pred_list)
    return accuracy, cm



In [5]:

# Uploading Training and Testing Files
csv_file_train = "details/train_feature.csv"
csv_file_test="details/test_feature.csv"
data_frame_train = pd.read_csv(csv_file_train)
data_frame_test = pd.read_csv(csv_file_test)

# Shuffling the data
train_df, test_df = shuffle_data(data_frame_train,data_frame_test)

# Making Custom Dataset
train_dataset = CustomDataset(train_df)
test_dataset = CustomDataset(test_df)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Initialization of Model
model = Model()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training Model
train_model(model, train_loader, test_loader)

# Evaluating Model
best_model = Model()
best_model.to(device)
best_model.load_state_dict(torch.load(pth_file_path))
test_accuracy, cm = evaluate_model(best_model, test_loader)
print(f"Final Test Accuracy: {test_accuracy}")



c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Epoch 1/50, Train Loss: 0.6412581740179031, Test Accuracy: 0.6171364700129239
Model saved!
Epoch 2/50, Train Loss: 0.6183947704796372, Test Accuracy: 0.600707906900928
Epoch 3/50, Train Loss: 0.6179006052535235, Test Accuracy: 0.6074027501251644
Epoch 4/50, Train Loss: 0.6176990644102957, Test Accuracy: 0.6120949619849337
Epoch 5/50, Train Loss: 0.6177346241608453, Test Accuracy: 0.6128866999662347
Epoch 6/50, Train Loss: 0.6176984837286718, Test Accuracy: 0.6042357981999604
Epoch 7/50, Train Loss: 0.6174616546653883, Test Accuracy: 0.6057028421064887
Epoch 8/50, Train Loss: 0.6166833632110015, Test Accuracy: 0.6007544797233574
Epoch 9/50, Train Loss: 0.6119779997821018, Test Accuracy: 0.6035488490691258
Epoch 10/50, Train Loss: 0.6046574579735682, Test Accuracy: 0.6008243389570016
Epoch 11/50, Train Loss: 0.600895140895644, Test Accuracy: 0.5897749368356096
Epoch 12/50, Train Loss: 0.5992261762910647, Test Accuracy: 0.5981696880785218
Epoch 13/50, Train Loss: 0.5977960069675814, Test 

In [6]:
# Save best_model as a file in ONNX format. ONNX is an open format used to represent deep learning models. It can be supported by a variety of deep learning frameworks (including PyTorch and ML.NET in .NET).
tensor_input = torch.randn(1, 140)  # Create a fake input tensor
torch.onnx.export(model, tensor_input, onnx_file_path, input_names=['input'], verbose=True) # Here you need to display the definition of input_names
print("The model was successfully exported as an ONNX file: ", onnx_file_path)

The model was successfully exported as an ONNX file:  ./eeg_emotion-best_model_2024-06-04.onnx
